## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_to_load = os.path.join("../Weather_Database/WeatherPy_database.csv")

city_data_df = pd.read_csv(city_data_to_load)
city_data_df


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,39.31,81,40,16.11,scattered clouds
1,1,Topolobampo,MX,25.6000,-109.0500,64.98,74,60,4.85,broken clouds
2,2,Alekseyevsk,RU,57.8406,108.3492,18.46,89,100,6.55,overcast clouds
3,3,Lebu,CL,-37.6167,-73.6500,52.54,78,0,19.98,clear sky
4,4,Kapaa,US,22.0752,-159.3190,82.38,71,40,19.57,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
690,690,Portage,US,41.1667,-81.1998,38.98,69,100,1.01,overcast clouds
691,691,Onega,RU,63.9061,38.1404,9.59,96,96,4.88,overcast clouds
692,692,Luanda,AO,-8.8368,13.2343,75.20,94,75,2.30,broken clouds
693,693,Aguimes,ES,27.9054,-15.4461,61.43,68,40,12.66,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What minimum temperature would you prefer for your trip? "))
max_temp = float(input("What maximum temperature would you prefer for your trip? "))



What minimum temperature would you prefer for your trip? 75
What maximum temperature would you prefer for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Kapaa,US,22.0752,-159.3190,82.38,71,40,19.57,scattered clouds
6,6,Ternate,ID,0.8000,127.4000,83.12,71,91,3.33,overcast clouds
11,11,Laguna,US,38.4210,-121.4238,78.78,32,0,11.50,clear sky
12,12,Puerto Ayora,EC,-0.7393,-90.3518,76.55,95,100,8.01,overcast clouds
13,13,Butaritari,KI,3.0707,172.7902,81.54,73,98,11.54,overcast clouds
19,19,Vaini,TO,-21.2000,-175.2000,78.96,89,75,6.91,broken clouds
21,21,Rikitea,PF,-23.1203,-134.9692,77.76,64,100,9.66,overcast clouds
25,25,Georgetown,MY,5.4112,100.3354,82.33,92,20,5.01,few clouds
26,26,Araouane,ML,18.9048,-3.5265,82.83,14,2,5.95,clear sky
38,38,Porto Novo,BJ,6.4965,2.6036,81.57,82,100,7.63,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

# there are no empty rows

City_ID                186
City                   186
Country                186
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Current Description    186
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna(how='all')

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,
6,Ternate,ID,83.12,overcast clouds,0.8000,127.4000,
11,Laguna,US,78.78,clear sky,38.4210,-121.4238,
12,Puerto Ayora,EC,76.55,overcast clouds,-0.7393,-90.3518,
13,Butaritari,KI,81.54,overcast clouds,3.0707,172.7902,
19,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
21,Rikitea,PF,77.76,overcast clouds,-23.1203,-134.9692,
25,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
26,Araouane,ML,82.83,clear sky,18.9048,-3.5265,
38,Porto Novo,BJ,81.57,overcast clouds,6.4965,2.6036,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Ternate,ID,83.12,overcast clouds,0.8000,127.4000,Sahid Bela Ternate Hotel
11,Laguna,US,78.78,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
12,Puerto Ayora,EC,76.55,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
13,Butaritari,KI,81.54,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
19,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
21,Rikitea,PF,77.76,overcast clouds,-23.1203,-134.9692,People ThankYou
25,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
26,Araouane,ML,82.83,clear sky,18.9048,-3.5265,
38,Porto Novo,BJ,81.57,overcast clouds,6.4965,2.6036,Tour Eiffel Hotel Benin


In [10]:
hotel_df.isnull().head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False
11,False,False,False,False,False,False,False
12,False,False,False,False,False,False,False
13,False,False,False,False,False,False,False
19,False,False,False,False,False,False,False
21,False,False,False,False,False,False,False
25,False,False,False,False,False,False,False
26,False,False,False,False,False,False,False
38,False,False,False,False,False,False,False


In [27]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(inplace=True)

clean_hotel_df = hotel_df
clean_hotel_df.count()

City                   174
Country                174
Max Temp               174
Current Description    174
Lat                    174
Lng                    174
Hotel Name             174
dtype: int64

In [29]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))